# portfolio optimization
- dataset preparation

In [37]:
import os
import numpy as np
import pandas as pd
from typing import Optional, List, Dict

import holidays

In [38]:
# path constant
data_root = "./data/"
raw = "raw"
# manual constants

# 다룰 데이터의 시작 지점
start_date = "2006-01-01"
end_date = "2021-12-31"
timelines = ["9008", "0924"]
timelines_alt = "0208"
window_size_years = 7
step_size_years = 1
data_split: List[int] = {
    "train": 5,
    "val": 1,
    "test": 1}  # train, val, test
default_key = "SPX"
# data structure

start_date: pd.Timestamp = pd.to_datetime(start_date)
end_date: pd.Timestamp = pd.to_datetime(end_date)

In [39]:
# code snippet
## 영업일 filter
def get_closed_days(start_date: pd.Timestamp, end_date: pd.Timestamp):
    us_holidays_dict = holidays.XNYS(years=range(start_date.year, end_date.year+1))  # 2002년부터 2023년까지의 공휴일
    us_holidays = pd.to_datetime(list(us_holidays_dict.keys()))
    
    all_dates = pd.date_range(start=start_date, end=end_date)
    weekends = all_dates[all_dates.weekday >= 5]  # 토요일(5), 일요일(6)

    # 공휴일과 주말 합치기
    all_holidays = pd.concat([pd.Series(us_holidays), pd.Series(weekends)]).drop_duplicates().sort_values()
    
    return all_holidays

def convert_volume(value):  # to handle Vol. Column
    if isinstance(value, str):
        value = value.strip()
        if value.endswith('B'):
            return float(value[:-1]) * 10**9
        elif value.endswith('M'):
            return float(value[:-1]) * 10**6
        elif value.endswith('K'):
            return float(value[:-1]) * 10**3
        elif value == '':
            return np.nan
        else:
            return float(value)
    else:
        return np.nan

# hard coded values    
def convert_columns(df: pd.DataFrame) -> pd.DataFrame:
    numeric_cols = ['Price', 'Open', 'High', 'Low']
    df[numeric_cols] = df[numeric_cols].map(lambda x: str(x).replace(',', '')).astype(float)

    df["Vol."] = df["Vol."].apply(convert_volume)

    df['Change %'] = df['Change %'].str.replace('%', '', regex=False).astype(float) * 0.01
        
    return df

In [40]:
filter_date: bool = True 
filter_holidays: bool = True

base_ticker: str = "SPX"
interpolation_target: List[str] = ["SPXHC"]

In [41]:
# raw data processing

snp_indices = {
    "SPX": "S&P 500",  # done
    "SPLRCT": "S&P 500 Information Technology",  # done
    "SPLRCL": "S&P 500 Telecom Services",  # done
    "SPLRCM": "S&P 500 Materials",  # done
    "SPLRCREC": "S&P 500 Real Estate",  # 20020101부터 사용가능  # done
    "SPLRCS": "S&P 500 Consumer Staples",  # done
    "SPSY": "S&P 500 Financials",  # done
    "SPNY": "S&P 500 Energy",  # done
    "SPXHC": "S&P 500 Health Care",  # done, # 중간에 정보 손실 interpolation 해버리기
    "SPLRCD": "S&P 500 Consumer Discretionary",  # done
    "SPLRCI": "S&P 500 Industrials",  # done
    "SPLRCU": "S&P 500 Utilities",  # done
    "VIX": "CBOE Volatility Index",  # done
}  # {ticker: name} for investing.com

snp_indices_raw_data = {}

for key, value in snp_indices.items():
    print(f"Raw data Processing {value}")
    temp_df_list = []
    for _each_timeline in timelines:
        # load csv
        if _each_timeline == "9008" and key == "SPLRCREC":
            each_timeline = timelines_alt
        else:
            each_timeline = _each_timeline
        csv_path = os.path.join(data_root, raw, f"{key}{each_timeline}.csv")    
        each_df = pd.read_csv(csv_path)
        each_df["Date"] = pd.to_datetime(each_df["Date"])
        
        # specific date range        
        if filter_date:
            each_df = each_df[(each_df["Date"] >= start_date) & (each_df["Date"] <= end_date)]

        temp_df_list.append(each_df)
        
    merged_df = pd.concat(temp_df_list)
    merged_df = merged_df.sort_values("Date", ascending=True)
    
    if filter_holidays:
        us_holidays = get_closed_days(start_date, end_date)
        # 2002년부터 2023년까지의 공휴일
        merged_df = merged_df[~merged_df["Date"].isin(us_holidays)]
        
    merged_df = convert_columns(merged_df)
        
    merged_df["T"] = range(len(merged_df))
    merged_df = merged_df.reset_index(drop=True).set_index("T")
    
    snp_indices_raw_data[key] = merged_df

if len(interpolation_target) > 0:
    print("Missing value linear interpolation")
    full_date = snp_indices_raw_data[base_ticker]["Date"]
    
    for key in interpolation_target:
        snp_indices_raw_data[key] = snp_indices_raw_data[key].set_index("Date").reindex(full_date)# .interpolate(method="linear")
        snp_indices_raw_data[key]["T"] = range(len(snp_indices_raw_data[key]))
        snp_indices_raw_data[key] = snp_indices_raw_data[key].reset_index(drop=False).set_index("T")


Raw data Processing S&P 500
Raw data Processing S&P 500 Information Technology
Raw data Processing S&P 500 Telecom Services
Raw data Processing S&P 500 Materials
Raw data Processing S&P 500 Real Estate
Raw data Processing S&P 500 Consumer Staples
Raw data Processing S&P 500 Financials
Raw data Processing S&P 500 Energy
Raw data Processing S&P 500 Health Care
Raw data Processing S&P 500 Consumer Discretionary
Raw data Processing S&P 500 Industrials
Raw data Processing S&P 500 Utilities
Raw data Processing CBOE Volatility Index
Missing value linear interpolation


In [42]:
snp_indices_raw_data["SPX"]# key # snp_indices_raw_data[key]

,Date,Price,Open,High,Low,Vol.,Change %
T,,,,,,,
0,2006-01-03,1268.8,1248.3,1270.2,1245.7,NaN,0.0164
1,2006-01-04,1273.5,1268.8,1275.4,1267.7,NaN,0.0037
2,2006-01-05,1273.5,1273.5,1276.9,1270.3,NaN,0.0000
3,2006-01-06,1285.5,1273.5,1286.1,1273.5,NaN,0.0094
4,2006-01-09,1290.2,1285.5,1290.8,1284.8,NaN,0.0037
...,...,...,...,...,...,...,...
4023,2021-12-27,4791.2,4734.0,4791.5,4734.0,1.450000e+09,0.0138
4024,2021-12-28,4786.4,4795.5,4807.0,4780.0,1.370000e+09,-0.0010
4025,2021-12-29,4793.1,4788.6,4804.1,4778.1,1.250000e+09,0.0014


In [43]:
for each_year in range(2006, 2023):
    print(f"Year {each_year}")
    print(f'num work date: {len(snp_indices_raw_data["SPX"][snp_indices_raw_data["SPX"]["Date"].dt.year == each_year])}')

Year 2006
num work date: 251
Year 2007
num work date: 251
Year 2008
num work date: 253
Year 2009
num work date: 252
Year 2010
num work date: 252
Year 2011
num work date: 252
Year 2012
num work date: 250
Year 2013
num work date: 252
Year 2014
num work date: 252
Year 2015
num work date: 252
Year 2016
num work date: 252
Year 2017
num work date: 251
Year 2018
num work date: 251
Year 2019
num work date: 252
Year 2020
num work date: 253
Year 2021
num work date: 252
Year 2022
num work date: 0


In [44]:
"""
sanity check
동일한 영업일에서 데이터가 형성되어있는 가를 확인하기 위함
SPXHC ticker에 대해서만 일부 데이터 누락이 있어서 interpolation이 필요함
기준은 SPX로 한다.

"""

# overall sanity check
all_dates = [df["Date"] for df in snp_indices_raw_data.values()]
reference_dates = all_dates[0]

for key, dates in zip(snp_indices_raw_data.keys(), all_dates):
    print(f"{key}: {len(dates)}")
    if not dates.equals(reference_dates):
        print(f"{key} has different dates.")
    else:
        print(f"{key} has the same dates.")

### ticker_wise check
search_target = "short_date" # "short_date"
target_ticker = "SPXHC"
short_date = snp_indices_raw_data[target_ticker]["Date"].to_list()

missing_dates = {}
for key, each_df in snp_indices_raw_data.items():
    if key != target_ticker:
        missing_dates[key] = [date for date in short_date if date not in each_df["Date"].values]

# 누락된 날짜 출력
for key, dates in missing_dates.items():
    print(f"{key} missing dates:")
    print(dates)

SPX: 4028
SPX has the same dates.
SPLRCT: 4028
SPLRCT has the same dates.
SPLRCL: 4028
SPLRCL has the same dates.
SPLRCM: 4028
SPLRCM has the same dates.
SPLRCREC: 4028
SPLRCREC has the same dates.
SPLRCS: 4028
SPLRCS has the same dates.
SPSY: 4028
SPSY has the same dates.
SPNY: 4028
SPNY has the same dates.
SPXHC: 4028
SPXHC has the same dates.
SPLRCD: 4028
SPLRCD has the same dates.
SPLRCI: 4028
SPLRCI has the same dates.
SPLRCU: 4028
SPLRCU has the same dates.
VIX: 4028
VIX has the same dates.
SPX missing dates:
[]
SPLRCT missing dates:
[]
SPLRCL missing dates:
[]
SPLRCM missing dates:
[]
SPLRCREC missing dates:
[]
SPLRCS missing dates:
[]
SPSY missing dates:
[]
SPNY missing dates:
[]
SPLRCD missing dates:
[]
SPLRCI missing dates:
[]
SPLRCU missing dates:
[]
VIX missing dates:
[]


In [62]:
def split_window_by_year(window: pd.DataFrame, split: Dict[str, int]) -> Dict[str, pd.DataFrame]:
    split_dfs = {}
    start_year = window['Date'].dt.year.min()
    
    for key, years in split.items():
        end_year = start_year + years
        split_dfs[key] = window[(window['Date'].dt.year >= start_year) & (window['Date'].dt.year < end_year)]
        start_year = end_year
    
    return split_dfs

def window_by_year(df_dict: Dict[str, pd.DataFrame], start_date: pd.Timestamp, window_size_years: int, split: Dict[str, int], is_last: bool = False, raw_end_date: Optional[pd.Timestamp]=None) -> Dict[str, pd.DataFrame]:
    folded_dict_df: Dict[str, pd.DataFrame] = {}
    
    if is_last:
        assert raw_end_date is not None, "raw_end_date is required"
        for key, each_df in df_dict.items():
                window_each_df = each_df[(each_df['Date'] >= start_date) & (each_df['Date'] < raw_end_date)]
                window_df_split_dict = split_window_by_year(window_each_df, split)
                folded_dict_df[key] = window_df_split_dict
    else:
        for key, each_df in df_dict.items():
                window_end_date = start_date + pd.DateOffset(years=window_size_years)
                window_each_df = each_df[(each_df['Date'] >= start_date) & (each_df['Date'] < window_end_date)]
                window_df_split_dict = split_window_by_year(window_each_df, split)
                folded_dict_df[key] = window_df_split_dict

    return folded_dict_df

def sliding_window_by_date(df_dict: Dict[str, pd.DataFrame], window_size_years: int, split: Dict[str, int], step_size_years: int, default_key: str):
    assert window_size_years == sum(split.values()), "Window size and split size mismatch"
    
    folded_snp_indices_raw_data: List[Dict[Dict[str, Dict[str, pd.DataFrames]]]] = []
    start_date = df_dict[default_key]['Date'].min()
    end_date = df_dict[default_key]['Date'].max()
    
    while start_date + pd.DateOffset(years=window_size_years) <= end_date:
        folded_dict_df = window_by_year(df_dict, start_date, window_size_years, split)
        start_date += pd.DateOffset(years=step_size_years)
        folded_snp_indices_raw_data.append(folded_dict_df)
    
    if start_date < end_date:
        folded_dict_df = window_by_year(df_dict, start_date, window_size_years, split, is_last=True, raw_end_date=end_date)
        folded_snp_indices_raw_data.append(folded_dict_df)
        
    return folded_snp_indices_raw_data


# 슬라이딩 윈도우 적용
windows_list_df_dict = sliding_window_by_date(snp_indices_raw_data, window_size_years, data_split, step_size_years, default_key)

# 다만, train, eval, test split으로 분리를 해야 하니까 다시 조금만 더 가공이 필요함


In [64]:
windows_list_df_dict[9] # windows_list_df_dict[8]

{'SPX': {'train':            Date   Price    Open    High     Low  Vol.  Change %
  T                                                              
  2266 2015-01-05  2020.6  2054.4  2054.4  2017.3   NaN   -0.0183
  2267 2015-01-06  2002.6  2022.2  2030.2  1992.4   NaN   -0.0089
  2268 2015-01-07  2025.9  2005.5  2029.6  2005.5   NaN    0.0116
  2269 2015-01-08  2062.1  2030.6  2064.1  2030.6   NaN    0.0179
  2270 2015-01-09  2044.8  2063.4  2064.4  2038.3   NaN   -0.0084
  ...         ...     ...     ...     ...     ...   ...       ...
  3518 2019-12-24  3223.4  3225.4  3226.4  3220.5   NaN   -0.0002
  3519 2019-12-26  3239.9  3227.2  3240.1  3227.2   NaN    0.0051
  3520 2019-12-27  3240.0  3247.2  3247.9  3234.4   NaN    0.0000
  3521 2019-12-30  3221.3  3240.1  3240.9  3216.6   NaN   -0.0058
  3522 2019-12-31  3230.8  3215.2  3231.7  3212.0   NaN    0.0029
  
  [1257 rows x 7 columns],
  'val':            Date   Price    Open    High     Low  Vol.  Change %
  T                    

In [ ]:
# sliding window by date

arr = np.arange(0, 4028)
window_size = 252*(5+1+1)
step_size = 252

def sliding_window_slicing(arr, window_size, step_size):
    num_windows = (len(arr) - window_size) // step_size + 1
    windows = []
    for i in range(num_windows):
        start = i * step_size
        end = start + window_size
        windows.append(arr[start:end])
    return windows

result = sliding_window_slicing(arr, window_size, step_size)
len(result)
result
# for win_idx in range(0, 4028, 252):
#     print(win_idx, win_idx + 252*5)

[array([   0,    1,    2, ..., 1761, 1762, 1763]),
 array([ 252,  253,  254, ..., 2013, 2014, 2015]),
 array([ 504,  505,  506, ..., 2265, 2266, 2267]),
 array([ 756,  757,  758, ..., 2517, 2518, 2519]),
 array([1008, 1009, 1010, ..., 2769, 2770, 2771]),
 array([1260, 1261, 1262, ..., 3021, 3022, 3023]),
 array([1512, 1513, 1514, ..., 3273, 3274, 3275]),
 array([1764, 1765, 1766, ..., 3525, 3526, 3527]),
 array([2016, 2017, 2018, ..., 3777, 3778, 3779])]

X: fold 방법으로 해야 겠다. 그리고, 5:1:1의 비율로 test하는 것으로 한다.

할 것 7년 단위 sampler implement해야 한다.
10개씩 window를 가지고 하는 방식으로,

나의 방법의 우월성을 보이려면 단순하게 가장 마지막 년도에 대하여, 좋은 성능을 보이면 된다.
data sample의 경우에 4000개 정도는 되서, 아마도 매우 작은 lora network 구현 하면 일단은 되긴 할 듯
prompt 기반으로 finetune하면 되기야 될 듯
